# **Análise de Segmentação de Clientes da ClickBus**

<h4 style="margin-bottom: 4px;"><strong>Objetivo do Projeto:</strong></h4>
<p style="margin-top: 0; font-size: 1rem;">
    Este projeto tem como objetivo segmentar a base de clientes da ClickBus, permitindo ações de marketing mais precisas, com foco em <strong>aumentar a retenção</strong> e o <strong>valor do cliente</strong>.
</p>


## **Importando as bibliotecas & Carregando os dados**
---

In [0]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [0]:
sns.set_style('whitegrid')

In [0]:
tabela_bronze = "estudo.default.tvendas_bronze"
tabela_cluster_silver = "estudo.default.tcluster_silver"

In [0]:

# Usando Spark para ler a tabela do metastore
df_spark = spark.table(tabela_bronze)

# Convertendo para pandas
df_raw = df_spark.toPandas()

# Verificando os tipos das colunas
df_raw.info()

In [0]:
# Definino uma seed totalmente aleatória
SEED = 65426158

## **Transformando e Agregando os dados**
---

### Análise **RFM**

| Métrica     | Descrição                         |
|-------------|-----------------------------------|
| **Recência** | Tempo desde a última compra       |
| **Frequência** | Quantidade de compras realizadas |
| **Monetário** | Valor total gasto pelo cliente   |


In [0]:
# Ensure date_purchase and time_purchase are strings
df_raw['date_purchase'] = df_raw['date_purchase'].astype(str)
df_raw['time_purchase'] = df_raw['time_purchase'].astype(str)

# Create the datetime_purchase column with complete date and time information
df_raw['datetime_purchase'] = pd.to_datetime(df_raw['date_purchase'] + ' ' + df_raw['time_purchase'])

# Calculate Recency by getting the most recent purchase date
snapshot_date = df_raw['datetime_purchase'].max() + pd.Timedelta(days=1)

print(f'The last purchase was made on {snapshot_date.date()} at {snapshot_date.strftime("%H:%M")}')

In [0]:
# Agrupando os dados por cliente e calculando as métricas

df_customers = df_raw.groupby('fk_contact').agg({
    'datetime_purchase': lambda date: (snapshot_date - date.max()).days, # Recência: Dias desde a última compra do cliente
    'nk_ota_localizer_id': 'count', # Frequência: Contar a quantidade de compras 
    'gmv_success': 'sum' # Monetário: Somar o valor de todas as compras
})

# Renomeando as colunas para ficar mais legível
df_customers.rename(columns={'datetime_purchase': 'Recency',
                             'nk_ota_localizer_id': 'Frequency',
                             'gmv_success': 'Monetary'}, inplace=True)

df_customers.head(3)

## **Normalizando as variável de _df_customers_**
---

In [0]:
# Selecionando as variáveis de treino
x = df_customers[['Recency', 'Frequency', 'Monetary']]

# Criando o objeto de normalização
scaler = StandardScaler()

# Treinando e normalizando os nossos dados
x_scaled = scaler.fit_transform(x)

x_scaled

In [0]:
# Passando esse array para um dataframe
x_scaled = pd.DataFrame(x_scaled, columns = x.columns, index = x.index)

# Amostra dos nossos dados RPM após normalização
x_scaled.sample(5, random_state=1542345)

## **Qual o número de clusters (categorias) ideal?**
---

- **Poucos clusters:**  
  Podem agrupar clientes muito diferentes no mesmo grupo, escondendo informações relevantes.

- **Muitos clusters:**  
  Podem criar grupos pequenos demais e difíceis de interpretar, aumentando a complexidade sem necessidade.

Utilizamos o **Método do Cotovelo (Elbow Method)** para encontrar o **ponto de equilíbrio ideal**, garantindo que os grupos corretamente separados


In [0]:
# Se os clientes estão próximos um do outro, a inércia é baixa.
# Já se eles estão longe / bagunçados, a inércia é alta.

# Vamos guardar o valor da "bagunça" gerado para cada quantidade de clusters, queremos uma inércia (bagunça) baixa.
inertia_list = []

# Iremos ver o nível de inércia para cada quantidade de clusters (1 a 12)
k_range = range(1, 11) 

for k in k_range:
    # Criamos o modelo K-Means para o cluster da vez
    kmeans = KMeans(n_clusters=k, random_state=SEED, n_init=10) # n_init=10 executa 10 vezes para encontrar a melhor clusterização e evitar azar.

    # Treinamos o modelo
    kmeans.fit(x_scaled)

    # Guardamos o valor da inércia
    inertia_list.append(kmeans.inertia_)

# Agora, vamos plotar o gráfico do cotovelo
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia_list, marker='o', linestyle='--')
plt.title('Método do Cotovelo (Elbow Method)')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia / Bagunça')
plt.xticks(k_range)
plt.show()

## **Treinando o modelo de acordo com o número ideial de clusters**
---

In [0]:
# Definindo o número ideal de clusters
ideal_k = 4

# Criando e treinando o modelo final
kmeans_final = KMeans(n_clusters=ideal_k, random_state=SEED, n_init=10)
kmeans_final.fit(x_scaled)

# Pegando o valor do cluster e adicionando no nosso dataframe
df_customers['Cluster'] = kmeans_final.labels_

# Podemos ver que cada fk_contact é atribuido a um cluster / grupo
df_customers.sample(5, random_state=SEED)

## **Analisando o Perfil de Cada Cluster**
---

### **Serão analisados três gráficos:**

1. **Recência vs Monetário**  
   Correlaciona os dias desde a última compra com o valor total gasto.

2. **Frequência vs Monetário**  
   Correlaciona a frequência de compras realizadas com o valor total gasto.

3. **Recência vs Frequência**  
   Correlaciona a frequência de compras realizadas com os dias desde a última compra.


In [0]:
plt.figure(figsize=(10, 7))

# Criamos o gráfico de dispersão
sns.scatterplot(
    data=df_customers,
    x='Recency',
    y='Monetary',
    hue='Cluster', 
    palette='deep',     
    s=80,                
    alpha=0.8            
)

plt.title('Dias desde a Última Compra X Valor Total Gasto', fontsize=16, fontweight='bold')
plt.xlabel('Dias')
plt.ylabel('Valor Total Gasto (R$)')
plt.show()

In [0]:
plt.figure(figsize=(10, 7))

# Criamos o gráfico de dispersão
sns.scatterplot(
    data=df_customers,
    x='Frequency',
    y='Monetary',
    hue='Cluster', 
    palette='deep',     
    s=80,                
    alpha=0.8            
)

plt.title('Frequência de Compras X Valor Total Gasto', fontsize=16, fontweight='bold')
plt.xlabel('Frequência de Compras')
plt.ylabel('Valor Total Gasto (R$)')
plt.show()

In [0]:
plt.figure(figsize=(10, 7))

# Criamos o gráfico de dispersão
sns.scatterplot(
    data=df_customers,
    x='Recency',
    y='Frequency',
    hue='Cluster', 
    palette='deep',     
    s=80,                
    alpha=0.8            
)

plt.title('Frequência de Compras X Dias desde a Última Compra', fontsize=16, fontweight='bold')
plt.xlabel('Dias')
plt.ylabel('Frequência de Compras')
plt.show()

## **Interpretação dos Clusters**

Com base na análise podemos propor as seguintes labels para os clusters:

| Cluster | Descrição            | Recência | Frequência | Valor Gasto |
|----------|---------------------|----------|------------|-------------|
| **0**    | **Clientes Novos**   | Alta   | Baixa  | Baixo    |
| **1**    | **Clientes Inativos**| Baixa    | Baixa  | Baixo    |
| **2**    | **Clientes Premium** | Alta   | Alta   | Alto    |
| **3**    | **Clientes Comuns**  | Média   | Média  | Médio    |





## **Analisando a distribuição por Cluster**
---

### Relacionando o cluster com a descrição & Renomeando as colunas

In [0]:
# Estamos relacionando o numero do cluster com a sua descrição
cluster_map = {
    0: 'Cliente Novo',
    1: 'Cliente Inativo',
    2: 'Cliente Premium',
    3: 'Cliente Comum'
}

# Criamos uma nova coluna 'Cluster_Label' no DataFrame
df_customers['Cluster_Label'] = df_customers['Cluster'].map(cluster_map)

# Renomeamos as colunas
df_customers = df_customers.rename(columns={
    'Recency': 'recencia_dias',
    'Frequency': 'frequencia_de_compras',
    'Monetary': 'valor_total_gasto',
    'Cluster': 'cluster',
    'Cluster_Label': 'segmento_do_cliente'
})

df_customers.head(2)

In [0]:
# Calculamos a distribuição de frequência para cada cluster
df_dist = df_customers['segmento_do_cliente'].value_counts(normalize=True).reset_index()
df_dist.columns = ['segmento_do_cliente', 'frequencia']  # Renomeia as colunas
df_dist = df_dist.replace([np.inf, -np.inf], np.nan).dropna(subset=['frequencia'])

plt.figure(figsize=(10, 7))

# Gráfico de barras
ax = sns.barplot(
    data=df_dist,
    x='segmento_do_cliente',
    y='frequencia',
    palette='deep'
)

# Adiciona rótulos
for p in ax.patches:
    height = p.get_height()
    if np.isfinite(height):  
        ax.text(
            x=p.get_x() + p.get_width() / 2, 
            y=height + 0.01, 
            s=f'{height*100:.2f}%',
            ha='center'
        )

plt.title('Distribuição de Frequência por Cluster', fontsize=16, fontweight='bold')
plt.xlabel('')
plt.ylabel('Frequência')
plt.ylim(0, df_dist['frequencia'].max() + 0.05)  
plt.show()

## **Resumo Estatístico dos Clusters**
---

In [0]:
# Agrupando os valores para obter um resumo estatístico de todas as categorias
df_analysis = df_customers.groupby('segmento_do_cliente').agg(
    Recencia_Media=('recencia_dias', 'mean'),
    Frequencia_Media=('frequencia_de_compras', 'mean'),
    Valor_Medio=('valor_total_gasto', 'mean'),
    Contagem_Clientes=('cluster', 'count')
).round(2)

# Ordenando do maior ao menor número de ocorrências
df_analysis.sort_values(by='Contagem_Clientes', ascending=False)

## **Conclusão sobre a Distribuição dos Clientes**
---

### Analisando os dados, percebemos um comportamento claro no público da **ClickBus**:

- **📈 67% dos clientes são novos**  
  Realizaram uma compra recentemente, mas ainda têm baixo valor agregado e pouca recorrência.

- **📉 31% dos clientes estão inativos**  
  Ou seja, não retornaram para novas compras após as primeiras interações.

- **🔎 Apenas 1,29% dos clientes são recorrentes ou de alto valor**  
  Isso inclui os **Clientes Comuns e Premium**, indicando baixa retenção e fidelização.

<br>

---


### **O que deve ser feito?**

O foco deve ser em:

- **Engajar os clientes novos**  
  Incentivar a recompra e criar relacionamento desde o primeiro contato.

- **Reativar os clientes inativos**  
  Utilizar campanhas específicas de recuperação e oferta direcionada.

- **Promover a evolução do cliente no ciclo de valor**  
  Levar o cliente de "novo" para "comum" ou "premium", de acordo com o potencial de compra e perfil de gasto.

<br>

---

## **Entrega Final:** _fk_conctact_ **+** _Cluster_Label_

#### Com essa tabela a **ClickBus** consegue obter uma profunda análise sobre cada cliente e melhorar sua tomada de decisões

In [0]:
# Pequena Amostra da tabela final
df_customers.sample(6, random_state=SEED)

In [0]:
df_customers.reset_index(inplace=True)

In [0]:
df_customers.head()